# Close The Loops

## Setup

In [ ]:
from tecton_gen_ai.testing import set_dev_mode, print_md

set_dev_mode()

task = "You are an assistant helping a user find a real estate property based on their preferences"
model = {"model": "openai/gpt-4o", "temperature": 0}

## The naive agent

In [ ]:
from tecton_gen_ai.api import Agent

agent = Agent(
    name="agent",
    prompt=task,
    llm=model,
)

### Test the agent

In [ ]:
print_md(agent.invoke("what properties are under 300k"))

## Let the agent access properties

In [ ]:
def get_candidates(
    min_budget: float,
    max_budget: float,
    min_area: float,
    max_area: float,
    min_bedrooms: int,
    max_bedrooms: int,
) -> list[dict]:
    """
    Get properties based on the given budget, bedroom count and area range

    Args:

        min_budget (float): Minimum budget, 0 if not specified
        max_budget (float): Maximum budget, 0 if not specified
        min_area (float): Minimum area, 0 if not specified
        max_area (float): Maximum area, 0 if not specified
        min_bedrooms (int): Minimum bedroom count, 0 if not specified
        max_bedrooms (int): Maximum bedroom count, 0 if not specified

    Returns:

        list[dict]: List of properties that match the given budget and room count range
    """
    properties = [
        {
            "id": 1,
            "price": 100000,
            "room_count": 2,
            "name": "Meadowbrook Estates",
            "area": 1000,
        },
        {
            "id": 2,
            "price": 220000,
            "room_count": 3,
            "name": "Pinecrest Villas",
            "area": 1500,
        },
        {
            "id": 3,
            "price": 280000,
            "room_count": 2,
            "name": "Oceanview Residences",
            "area": 2200,
        },
        {
            "id": 4,
            "price": 400000,
            "room_count": 5,
            "name": "Maplewood Manor",
            "area": 2000,
        },
        {
            "id": 5,
            "price": 500000,
            "room_count": 3,
            "name": "Riverstone Heights",
            "area": 1800,
        },
    ]
    filtered_properties = [
        p
        for p in properties
        if (min_budget == 0 or p["price"] >= min_budget)
        and (max_budget == 0 or p["price"] <= max_budget)
        and (min_area == 0 or p["area"] >= min_area)
        and (max_area == 0 or p["area"] <= max_area)
        and (min_bedrooms == 0 or p["room_count"] >= min_bedrooms)
        and (max_bedrooms == 0 or p["room_count"] <= max_bedrooms)
    ]
    return filtered_properties

In [ ]:
agent = Agent(
    name="agent",
    prompt=task,
    tools=[get_candidates],
    llm=model,
)

print_md(agent.invoke("what properties are under 300k?"))

## Let the agent read user preferences

See the user preference features [in the UI](https://dev-gen-ai.tecton.ai/app/repo/han-stream-1/home/dataflow)

In [ ]:
from tecton_gen_ai.api import FeatureServiceConfig

user_preference = FeatureServiceConfig(
    service="user_preference",
    workspace="han-stream-1",
)

In [ ]:
agent = Agent(
    name="agent",
    prompt=task,
    tools=[get_candidates, user_preference],
    llm=model,
)

agent.invoke_tool("han-stream-1_user_preference", kwargs={"user_id": "john"})

In [ ]:
print_md(agent.invoke("what properties do you recommend", context={"user_id": "john"}))

In [ ]:
from tecton_gen_ai.testing.interactive import chat

chat(agent, context={"user_id": "john"})

## Let the agent update user preference while chatting

In [ ]:
agent = Agent(
    name="agent",
    prompt=task,
    tools=[get_candidates, user_preference],
    emitters=[user_preference],
    llm=model,
)

In [ ]:
user_id = "user22"
agent.invoke_tool("han-stream-1_user_preference", kwargs={"user_id": user_id})

In [ ]:
chat(agent, context={"user_id": user_id})